# Before 9:30

In [10]:
import pandas as pd


# instruments_df = pd.read_csv('DataSets/example-set/input_instruments.csv')
# orders_df =  pd.read_csv('DataSets/example-set/input_orders.csv')
# clients_df = pd.read_csv('DataSets/example-set/input_clients.csv')


instruments_df = pd.read_csv('DataSets/test-set/input_instruments.csv')
orders_df =  pd.read_csv('DataSets/test-set/input_orders.csv')
clients_df = pd.read_csv('DataSets/test-set/input_clients.csv')

clients_df.set_index('ClientID', inplace=True)
orders_df['Rating'] = orders_df['Client'].map(clients_df['Rating'])
orders_df['Currencies'] = orders_df['Client'].map(clients_df['Currencies'])
orders_df['PositionCheck'] = orders_df['Client'].map(clients_df['PositionCheck'])

instruments_df.set_index('InstrumentID', inplace=True)
orders_df['LotSize'] = orders_df['Instrument'].map(instruments_df['LotSize'])

clients_df.reset_index(inplace=True)
instruments_df.reset_index(inplace=True)

orders_df



,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize
0,09:00:00,ORDER_10,INST_000,17500.0,CLIENT_107,Market,Sell,7,JPY,Y,1000
1,09:00:00,ORDER_1430,INST_000,3000.0,CLIENT_111,Market,Sell,6,"SGD,JPY,USD",Y,1000
2,09:00:00,ORDER_1860,INST_000,46000.0,CLIENT_113,Market,Sell,4,SGD,Y,1000
3,09:00:00,ORDER_20,INST_001,4350.0,CLIENT_106,Market,Sell,4,JPY,Y,100
4,09:00:00,ORDER_2430,INST_001,8700.0,CLIENT_103,Market,Buy,8,"SGD,USD,JPY",N,100
...,...,...,...,...,...,...,...,...,...,...,...
9995,16:09:00,ORDER_8859,INST_007,62.0,CLIENT_104,99.6,Sell,9,"SGD,JPY,USD",Y,1
9996,16:09:00,ORDER_9429,INST_008,59000.0,CLIENT_112,100.1,Buy,4,"JPY,USD,SGD",Y,1000
9997,16:09:00,ORDER_9859,INST_008,94000.0,CLIENT_109,100.9,Buy,3,"USD,SGD,JPY",Y,1000
9998,16:09:00,ORDER_10429,INST_009,31000.0,CLIENT_105,100.1,Sell,3,"JPY,USD,SGD",Y,1000


In [11]:
def validate_instruments(instruments_df, orders_df):
    instrument_ids_set = set(instruments_df['InstrumentID'])
    orders_df['Instrument_Valid'] = orders_df['Instrument'].isin(instrument_ids_set)

    orders_df['RejectionReason'] = ''

    orders_df.loc[orders_df['Instrument_Valid'] == False, 'RejectionReason'] = 'REJECTED - INSTRUMENT NOT FOUND'
    return orders_df

validate_instruments(instruments_df, orders_df)
orders_df

,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize,Instrument_Valid,RejectionReason
0,09:00:00,ORDER_10,INST_000,17500.0,CLIENT_107,Market,Sell,7,JPY,Y,1000,True,
1,09:00:00,ORDER_1430,INST_000,3000.0,CLIENT_111,Market,Sell,6,"SGD,JPY,USD",Y,1000,True,
2,09:00:00,ORDER_1860,INST_000,46000.0,CLIENT_113,Market,Sell,4,SGD,Y,1000,True,
3,09:00:00,ORDER_20,INST_001,4350.0,CLIENT_106,Market,Sell,4,JPY,Y,100,True,
4,09:00:00,ORDER_2430,INST_001,8700.0,CLIENT_103,Market,Buy,8,"SGD,USD,JPY",N,100,True,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,16:09:00,ORDER_8859,INST_007,62.0,CLIENT_104,99.6,Sell,9,"SGD,JPY,USD",Y,1,True,
9996,16:09:00,ORDER_9429,INST_008,59000.0,CLIENT_112,100.1,Buy,4,"JPY,USD,SGD",Y,1000,True,
9997,16:09:00,ORDER_9859,INST_008,94000.0,CLIENT_109,100.9,Buy,3,"USD,SGD,JPY",Y,1000,True,
9998,16:09:00,ORDER_10429,INST_009,31000.0,CLIENT_105,100.1,Sell,3,"JPY,USD,SGD",Y,1000,True,


In [12]:
def validate_currency(instruments_df, orders_df):
    currency_set = set(instruments_df['Currency'])

    for index, order in orders_df.iterrows():
        if not order['RejectionReason']:
            client_currencies = set(order['Currencies'].split(','))
            is_valid = any(currency in currency_set for currency in client_currencies)
            orders_df.at[index, 'Currency_Valid'] = is_valid

    orders_df.loc[orders_df['Currency_Valid'] == False, 'RejectionReason'] = 'REJECTED - MISMATCH CURRENCY'

    return orders_df

validate_currency(instruments_df, orders_df)


,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize,Instrument_Valid,RejectionReason,Currency_Valid
0,09:00:00,ORDER_10,INST_000,17500.0,CLIENT_107,Market,Sell,7,JPY,Y,1000,True,,True
1,09:00:00,ORDER_1430,INST_000,3000.0,CLIENT_111,Market,Sell,6,"SGD,JPY,USD",Y,1000,True,,True
2,09:00:00,ORDER_1860,INST_000,46000.0,CLIENT_113,Market,Sell,4,SGD,Y,1000,True,,True
3,09:00:00,ORDER_20,INST_001,4350.0,CLIENT_106,Market,Sell,4,JPY,Y,100,True,,True
4,09:00:00,ORDER_2430,INST_001,8700.0,CLIENT_103,Market,Buy,8,"SGD,USD,JPY",N,100,True,,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,16:09:00,ORDER_8859,INST_007,62.0,CLIENT_104,99.6,Sell,9,"SGD,JPY,USD",Y,1,True,,True
9996,16:09:00,ORDER_9429,INST_008,59000.0,CLIENT_112,100.1,Buy,4,"JPY,USD,SGD",Y,1000,True,,True
9997,16:09:00,ORDER_9859,INST_008,94000.0,CLIENT_109,100.9,Buy,3,"USD,SGD,JPY",Y,1000,True,,True
9998,16:09:00,ORDER_10429,INST_009,31000.0,CLIENT_105,100.1,Sell,3,"JPY,USD,SGD",Y,1000,True,,True


In [13]:
def validate_lotsize(orders_df):
  for i in range(0, len(orders_df)):
    if not orders_df.loc[i, 'RejectionReason']:
      if (orders_df.loc[i, 'Quantity'] % orders_df.loc[i, 'LotSize']) == 0:
        orders_df.loc[i, 'Lotsize_valid'] = True
      else:
        orders_df.loc[i, 'Lotsize_valid'] = False
        orders_df.loc[i, 'RejectionReason'] = "REJECTED - INVLALID LOT SIZE"

  return orders_df

validate_lotsize(orders_df)

,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize,Instrument_Valid,RejectionReason,Currency_Valid,Lotsize_valid
0,09:00:00,ORDER_10,INST_000,17500.0,CLIENT_107,Market,Sell,7,JPY,Y,1000,True,REJECTED - INVLALID LOT SIZE,True,False
1,09:00:00,ORDER_1430,INST_000,3000.0,CLIENT_111,Market,Sell,6,"SGD,JPY,USD",Y,1000,True,,True,True
2,09:00:00,ORDER_1860,INST_000,46000.0,CLIENT_113,Market,Sell,4,SGD,Y,1000,True,,True,True
3,09:00:00,ORDER_20,INST_001,4350.0,CLIENT_106,Market,Sell,4,JPY,Y,100,True,REJECTED - INVLALID LOT SIZE,True,False
4,09:00:00,ORDER_2430,INST_001,8700.0,CLIENT_103,Market,Buy,8,"SGD,USD,JPY",N,100,True,,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,16:09:00,ORDER_8859,INST_007,62.0,CLIENT_104,99.6,Sell,9,"SGD,JPY,USD",Y,1,True,,True,True
9996,16:09:00,ORDER_9429,INST_008,59000.0,CLIENT_112,100.1,Buy,4,"JPY,USD,SGD",Y,1000,True,,True,True
9997,16:09:00,ORDER_9859,INST_008,94000.0,CLIENT_109,100.9,Buy,3,"USD,SGD,JPY",Y,1000,True,,True,True
9998,16:09:00,ORDER_10429,INST_009,31000.0,CLIENT_105,100.1,Sell,3,"JPY,USD,SGD",Y,1000,True,,True,True


In [14]:
def validate_position_before_930(orders_df):
    for index, order in orders_df.iterrows():
        if not order['RejectionReason']:
            if order['PositionCheck'] == 'Y' and order['Side'] == 'Sell':
                orders_df.at[index, 'Position_Valid'] = False
            else:
                orders_df.at[index, 'Position_Valid'] = True

    orders_df.loc[orders_df['Position_Valid'] == False, 'RejectionReason'] = 'REJECTED - POSITION CHECK FAILED'

    return orders_df

validate_position_before_930(orders_df)

,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize,Instrument_Valid,RejectionReason,Currency_Valid,Lotsize_valid,Position_Valid
0,09:00:00,ORDER_10,INST_000,17500.0,CLIENT_107,Market,Sell,7,JPY,Y,1000,True,REJECTED - INVLALID LOT SIZE,True,False,NaN
1,09:00:00,ORDER_1430,INST_000,3000.0,CLIENT_111,Market,Sell,6,"SGD,JPY,USD",Y,1000,True,REJECTED - POSITION CHECK FAILED,True,True,False
2,09:00:00,ORDER_1860,INST_000,46000.0,CLIENT_113,Market,Sell,4,SGD,Y,1000,True,REJECTED - POSITION CHECK FAILED,True,True,False
3,09:00:00,ORDER_20,INST_001,4350.0,CLIENT_106,Market,Sell,4,JPY,Y,100,True,REJECTED - INVLALID LOT SIZE,True,False,NaN
4,09:00:00,ORDER_2430,INST_001,8700.0,CLIENT_103,Market,Buy,8,"SGD,USD,JPY",N,100,True,,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,16:09:00,ORDER_8859,INST_007,62.0,CLIENT_104,99.6,Sell,9,"SGD,JPY,USD",Y,1,True,REJECTED - POSITION CHECK FAILED,True,True,False
9996,16:09:00,ORDER_9429,INST_008,59000.0,CLIENT_112,100.1,Buy,4,"JPY,USD,SGD",Y,1000,True,,True,True,True
9997,16:09:00,ORDER_9859,INST_008,94000.0,CLIENT_109,100.9,Buy,3,"USD,SGD,JPY",Y,1000,True,,True,True,True
9998,16:09:00,ORDER_10429,INST_009,31000.0,CLIENT_105,100.1,Sell,3,"JPY,USD,SGD",Y,1000,True,REJECTED - POSITION CHECK FAILED,True,True,False


In [16]:
import csv


def generate_exchange_report(orders_df):
  with open('DataSets/exchange_report.csv', 'w', newline='') as file:
    headerList = ['OrderID', 'RejectionReason']
    dw = csv.DictWriter(file, delimiter=',',  
                        fieldnames=headerList) 
    dw.writeheader() 
    writer = csv.writer(file)
    for i in range(0, len(orders_df)):
      if orders_df.loc[i, 'RejectionReason'] != '':
        writer.writerow([orders_df.loc[i, 'OrderID'], orders_df.loc[i, 'RejectionReason']])
generate_exchange_report(orders_df)

In [96]:
orders_df.to_csv('DataSets/intermediate-set/orders_df_after_validation.csv', index=False)

...........

# New Trade

In [97]:
data = {
    'Time': ['09:30:01'],
    'Client': ['C'],
    'Instrument': ['SIA'],
    'Side': ['Sell'],
    'Price': ['Market'],
    'Quantity': [100],
    'OrderID': ['C2']
}

new_trade = pd.DataFrame(data)
prev_orders = pd.read_csv('DataSets/intermediate-set/previous-orders.csv')

new_trade


,Time,Client,Instrument,Side,Price,Quantity,OrderID
0,09:30:01,C,SIA,Sell,Market,100,C2


In [98]:
validate_instruments(instruments_df, new_trade)


,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason
0,09:30:01,C,SIA,Sell,Market,100,C2,True,


In [99]:
new_trade_currency = clients_df[clients_df['ClientID'] == new_trade['Client'].iloc[0]]['Currencies'].iloc[0]
new_trade['Currencies'] = new_trade_currency
new_trade

,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies
0,09:30:01,C,SIA,Sell,Market,100,C2,True,,SGD


In [100]:
validate_currency(instruments_df, new_trade)

,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies,Currency_Valid
0,09:30:01,C,SIA,Sell,Market,100,C2,True,,SGD,True


In [101]:
new_trade_lotsize = instruments_df[instruments_df['InstrumentID'] == new_trade['Instrument'].iloc[0]]['LotSize'].iloc[0]
new_trade['LotSize'] = new_trade_lotsize
new_trade

,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies,Currency_Valid,LotSize
0,09:30:01,C,SIA,Sell,Market,100,C2,True,,SGD,True,100


In [102]:
validate_lotsize(new_trade)

,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies,Currency_Valid,LotSize,Lotsize_valid
0,09:30:01,C,SIA,Sell,Market,100,C2,True,,SGD,True,100,True


In [103]:
import pandas as pd


def validate_position_continuous(new_trade, previous_orders):
    starting_letters = set(order[0] for order in previous_orders['SellOrder#'])
    if not any(new_trade['Client'].iloc[0].startswith(letter) for letter in starting_letters):
        if new_trade['Quantity'].iloc[0] < previous_orders['AskQty'].iloc[0]:
            new_trade.loc[0, 'Lotsize_valid'] = False
            new_trade.loc[0, 'RejectionReason'] = 'REJECTED - POSITION CHECK FAILED'
                
    return new_trade

validate_position_continuous(new_trade, prev_orders)


,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies,Currency_Valid,LotSize,Lotsize_valid
0,09:30:01,C,SIA,Sell,Market,100,C2,True,REJECTED - POSITION CHECK FAILED,SGD,True,100,False


In [104]:
rejected = pd.DataFrame(columns=['OrderID', 'RejectionReason']) 
if new_trade['RejectionReason'].iloc[0] != '':
    rejected = rejected.append({'OrderID': new_trade['OrderID'].iloc[0], 'RejectionReason': new_trade['RejectionReason'].iloc[0]}, ignore_index=True)
rejected


/var/folders/m_/kb9mf0b90hb2fvtfccyd41fr0000gn/T/ipykernel_86480/4274313354.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rejected = rejected.append({'OrderID': new_trade['OrderID'].iloc[0], 'RejectionReason': new_trade['RejectionReason'].iloc[0]}, ignore_index=True)


,OrderID,RejectionReason
0,C2,REJECTED - POSITION CHECK FAILED
